In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
import gc
import os
import json
from pathlib import Path
from os.path import join as path_join
import matplotlib.pyplot as plt
import numpy as np
import re

In [4]:
production_path = Path("../dataset_v1/data/tglang_dataset/Production_20000.parquet")

In [5]:
dataset = pd.read_parquet(production_path)[["tglang", "code", "target"]]
dataset.head(3)

,tglang,code,target
0,TGLANG_LANGUAGE_C,matching::computer_supporting_value<1>::type\n...,1
1,TGLANG_LANGUAGE_C,* @file rail_integration.c\n * @brief Simple c...,1
2,TGLANG_LANGUAGE_C,// Copyright (c) 2011-2015 The Pizcoin Core de...,1


In [6]:
target_2_lang = {}
for idx, row in dataset[["target", "tglang"]].drop_duplicates().iterrows():
    target_2_lang[row["target"]] = row["tglang"]

In [19]:
"""
    
"""
from pathlib import Path

keywords_root = Path("../dataset_v1/data/raw/keywords2")
keywords = []

for tglang_keywords in keywords_root.iterdir():
    tglang = tglang_keywords.name.split(".")[0]
    with open(tglang_keywords) as f:
        keywords += list(map(lambda x: x.strip(), f.readlines()))

keywords = list(set(keywords))
keywords_set = set(keywords)

for keyword in keywords_set:
    print(keyword)

modifier
orphans
connection_requests
ulong
paint
bytes_sent
NULL
sourceLocation
session_time
retry
fit
__virtual_inheritance
cursor
equals
aspect
geoip_city_country_code3
raise
before
True
consteval
__pin
all
geoip_latitude
__sptre
arguments
void
overscroll
ubyte
session_log_binary_id
scrollbar
CHECK
like
debugger
become
form
limit_rate
umask
actual
base
otel_trace_id
unsafe
connections_waiting
__unhook
include
decimal
__hook
top
gcnew
items
COLUMN
catch
uinteger
annotation
WORKDIR
interface
transition
object
dir
_ENCODING_
endfor
__based
option
into
geoip_org
__int32
primary
boolean
upstream_bytes_received
wbr
uint64
WHERE
upstream_trailer_
uri
grid
var
lua_isuserdata
lua_concat
static_assert
mask
http3
ssl_session_id
AND
selector
h2
frameset
punctuation
__stdcall
read
bytes_received
scoped
content_length
request_method
fn
strictfp
col
dim
csbyte
samp
while
fileID
foreach
scroll
dynamicparam
lua_insert
SET
ssl_session_reused
lua_atpanic
mark
partial
meter
search
__fastcall
__m128d
con

In [8]:
def my_tokenizer(s):
    single_token_ids  = set([9,10,32,34,39,40,41,44,59,91,93,96,123,125])
    word_ids          = set(list(range(65, 91)) + list(range(97, 123)) + [95])
    number_ids        = set(list(range(48, 58)))
    symbol_ids        = set([33,35,36,37,38,42,43,45,46,47,58,60,61,62,63,64,92,94,95,124,126])
    
    tokens = []
    cache = []
    token_type = 0
    
    for idx, c in enumerate(s):
        id = ord(c)
    
        if id in single_token_ids:

            if len(cache) > 0 and (token_type == 1 or token_type == 2 or token_type == 3):
                tokens.append("".join(cache))
            cache = []
            tokens.append(c)
            token_type = 0

        elif id in word_ids or (token_type == 1 and id in number_ids):

            if token_type == 0:
                cache = [c]
            elif token_type == 1:
                cache.append(c)
            elif token_type == 2:
                tokens.append("".join(cache))
                cache = [c]
            elif token_type == 3:
                tokens.append("".join(cache))
                cache = [c]
            token_type = 1

        elif id in symbol_ids:

            if token_type == 0:
                cache = [c]
            elif token_type == 1:
                tokens.append("".join(cache))
                cache = [c]
            elif token_type == 2:
                cache.append(c)
            elif token_type == 3:
                tokens.append("".join(cache))
                cache = [c]
            token_type = 2

        elif id in number_ids:

            if token_type == 0:
                cache = [c]
            elif token_type == 1:
                tokens.append("".join(cache))
                cache = [c]
            elif token_type == 2:
                tokens.append("".join(cache))
                cache = [c]
            elif token_type == 3:
                cache.append(c)
            token_type = 3

        else:
            if len(cache) > 0 and (token_type == 1 or token_type == 2 or token_type == 3):
                tokens.append("".join(cache))
            cache = []
            token_type = 0

        # print(c, id, cache)

    if len(cache) > 0 and (token_type == 1 or token_type == 2 or token_type == 3):
        tokens.append("".join(cache))

    return tokens

In [9]:
def char2num(c):
    n = ord(c)
    if n == 10:
        return 0
    elif  n <= 31 or n >= 127:
        return 96
    else:
        return n - 31


def get_embedding(s: str):
    if len(s) == 0:
        return np.zeros((96, ))
    res = np.array(np.bincount([char2num(c) for c in s], minlength=97), dtype=float) / len(s)
    return res[:-1]

In [10]:
def word2token_new(word: str):
    if word.isdigit():
        return '0'
    if (word in keywords_set) or re.search(r'[^a-zA-Z0-9_]', word):
        return word
    return ''.join([
        'A' if word[0].isupper() else ('a' if word[0].islower() else '_'),  # startswith 'A', 'a' or '_'
        'a' if any(map(str.islower, word[1:])) else '',
        'A' if any(map(str.isupper, word[1:])) else '',
        '_' if '_' in word else '',
        '2' if len(word) > 10 else ('0' if len(word) == 1 else "1")
    ])

def identity_tokenizer(text):
    return text

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_embeddings(train_code, train_target, tf_idf_params=None):
    if tf_idf_params is None:
        tf_idf_params = {
            "tokenizer": identity_tokenizer,
            "max_features": 1500,
            "lowercase": False
        }

    """
        Enrich train
    """

    train_tokens = map(my_tokenizer, train_code)
    train_tokens = map(
        lambda x: list(map(word2token_new, x)),
        train_tokens
    )
    train_data = list(filter(
        lambda x: len(x[0]) > 0,
        zip(train_tokens, train_target, train_code)
    ))

    train_tokens = list(map(lambda x: x[0], train_data))
    train_target = list(map(lambda x: x[1], train_data))
    train_code   = list(map(lambda x: x[2], train_data))

    """
        Vectorize
    """
    vectorizer = TfidfVectorizer(**tf_idf_params)
    vectorizer.fit(train_tokens)

    train_embeddings = vectorizer.transform(train_tokens)

    return train_embeddings, train_target, vectorizer

In [12]:
def train_forest(X_train, y_train):
    from sklearn.ensemble import RandomForestClassifier

    manual_best_params = {
        'criterion': 'gini',
        'max_features': 'sqrt',
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'n_estimators': 300,
        'max_depth': 24
    }

    clf = RandomForestClassifier(n_jobs=4, **manual_best_params)
    clf.fit(X_train, y_train)

    return clf

In [13]:
X_train, y_train, tfidf = get_embeddings(dataset["code"], dataset["target"])

/Users/platon.fedorov/opt/anaconda3/envs/tgml/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [14]:
forest = train_forest(X_train, y_train)

In [16]:
import joblib

joblib.dump(forest, "deploy_forest_20000s_1500f_300t_24d.pkl")
joblib.dump(tfidf, "deploy_tfidfi_20000s_1500f.pkl")

['deploy_tfidfi_20000s_1500f.pkl']

In [18]:
# code = """
# X_train, y_train, tfidf = get_embeddings(dataset["code"], dataset["target"])
# """

# code_embedding = tfidf.transform([list(map(word2token_new, my_tokenizer(code)))])
# (target_2_lang[forest.predict(code_embedding)[0]])

'TGLANG_LANGUAGE_PYTHON'